## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
##
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
##
import time
import random
from datetime import datetime

#visulaization modules
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Historic Data

In [2]:
data_old = pd.read_csv('export_GDPR_fines_last.csv')

In [3]:
data_old.head()

,Unnamed: 0,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type
0,0,ETid-1542,ITALY,2022-11-10,"20,000",Sportitalia,"Art. 5 (1) a) GDPR, Art. 9 GDPR, Art. 13 GDPR,...",Non-compliance with general data processing pr...
1,1,ETid-1541,ITALY,2022-11-24,"1,000,000",Areti spa,"Art. 5 (1) d), e) GDPR, Art. 5 (2) GDPR, Art. ...",Non-compliance with general data processing pr...
2,2,ETid-1540,ROMANIA,2022-12-27,"3,000",Kaufland Romania SCS,"Art. 29 GDPR, Art. 32 (1) b) GDPR, Art. 32 (2)...",Insufficient technical and organisational meas...
3,3,ETid-1539,ROMANIA,2022-12-22,"10,000",SUDREZIDENȚIAL Broker S.R.L.,Art. 32 (4) GDPR,Insufficient technical and organisational meas...
4,4,ETid-1538,ITALY,2022-11-10,"4,000",Villafranca di Verona municipality,"Art. 5 (1) a), c) GDPR, Art. 6 GDPR, Art. 2-te...",Non-compliance with general data processing pr...


## Scraping data

In [4]:
driver = webdriver.Chrome()
driver.get("https://www.enforcementtracker.com/")
driver.maximize_window()

In [5]:
#getting last ID vs historic data
last_current_id = driver.find_element(By.XPATH, '//*[@id="penalties"]/tbody/tr[1]/td[2]/a').text
last_current_id = last_current_id.replace('ETid-', '')

last_historic_id = data_old.loc[0]['ID'].replace('ETid-', '')

delta = int(last_current_id) - int(last_historic_id)

pages_to_load = delta//100 + 1

In [6]:
select = Select(driver.find_element(By.XPATH, '//*[@id="penalties_length"]/label/select'))
# select by visible text
select.select_by_visible_text('100')

In [7]:
headers = []
for i in range(2,14):
    elem = driver.find_element(By.XPATH, '//*[@id="penalties"]/thead/tr[1]/th['+str(i)+']')
    if elem.text != "":
        headers.append(elem.text)

In [8]:
data = []
content = []

for page in range(1, pages_to_load + 1):

    print('Page ' + str(page) + ' / ' + str(pages_to_load) + ' being scrapped.')
    
    for i in range(1,101):
        
        for j in range(2,13):
            
            elem = driver.find_element(By.XPATH, '//*[@id="penalties"]/tbody/tr['+str(i)+']/td['+str(j)+']')
            
            if elem.text != '':
                content.append(elem.text)
    
        data.append(content)
        content = []
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
    ActionChains(driver).click(driver.find_element(By.XPATH, '//*[@id="penalties_paginate"]/span/a['+str(page+1)+']')).perform()        
    
    print("Page " + str(page) + ' / ' + str(pages_to_load) + " done !")

    time.sleep(random.randint(2, 7))

Page 1 / 5 being scrapped.
Page 1 / 5 done !
Page 2 / 5 being scrapped.
Page 2 / 5 done !
Page 3 / 5 being scrapped.
Page 3 / 5 done !
Page 4 / 5 being scrapped.
Page 4 / 5 done !
Page 5 / 5 being scrapped.


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="penalties_paginate"]/span/a[6]"}
  (Session info: chrome=114.0.5735.248); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00DBA813+48355]
	(No symbol) [0x00D4C4B1]
	(No symbol) [0x00C55358]
	(No symbol) [0x00C809A5]
	(No symbol) [0x00C80B3B]
	(No symbol) [0x00CAE232]
	(No symbol) [0x00C9A784]
	(No symbol) [0x00CAC922]
	(No symbol) [0x00C9A536]
	(No symbol) [0x00C782DC]
	(No symbol) [0x00C793DD]
	GetHandleVerifier [0x0101AABD+2539405]
	GetHandleVerifier [0x0105A78F+2800735]
	GetHandleVerifier [0x0105456C+2775612]
	GetHandleVerifier [0x00E451E0+616112]
	(No symbol) [0x00D55F8C]
	(No symbol) [0x00D52328]
	(No symbol) [0x00D5240B]
	(No symbol) [0x00D44FF7]
	BaseThreadInitThunk [0x76CF7D59+25]
	RtlInitializeExceptionChain [0x775BB79B+107]

## Loading Data

In [9]:
df_delta = pd.DataFrame(data, columns = ['ID','Country','Date','Amount','Controller_Processor','Quoted_Art','Type','Source'])

In [10]:
df_delta.shape

(500, 8)

In [12]:
display(df_delta.head())
display(df_delta.info())

,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type,Source
0,ETid-1968,ROMANIA,2023-07-18,"3,000",ING Bank NV Amsterdam Sucursala București,"Art. 32 (1) b) GDPR, Art. 32 (2), (4) GDPR",Insufficient technical and organisational meas...,link
1,ETid-1967,SPAIN,2023-07-12,"10,000",Private individual,Art. 6 GDPR,Insufficient legal basis for data processing,link
2,ETid-1966,SPAIN,2023-07-16,300,Private individual,Art. 13 GDRP,Insufficient fulfilment of information obligat...,link
3,ETid-1965,IRELAND,2023-06-16,"1,000,000",Irish Departement of Health,"Art. 5 (1) c) GDPR, Art. 6 (1), (4) GDPR, Art....",Non-compliance with general data processing pr...,link
4,ETid-1964,SPAIN,2023-07-17,"10,000","NANDIVALE, S.L",Art. 6 (1) GDPR,Insufficient legal basis for data processing,link


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    500 non-null    object
 1   Country               500 non-null    object
 2   Date                  500 non-null    object
 3   Amount                500 non-null    object
 4   Controller_Processor  500 non-null    object
 5   Quoted_Art            500 non-null    object
 6   Type                  500 non-null    object
 7   Source                499 non-null    object
dtypes: object(8)
memory usage: 31.4+ KB


None

In [13]:
total_df = pd.concat([df_delta.drop('Source', axis = 1), data_old.drop(['Unnamed: 0'], axis = 1)], ignore_index=True)

total_df = total_df.drop_duplicates(subset = 'ID')

total_df.head()

,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type
0,ETid-1968,ROMANIA,2023-07-18,"3,000",ING Bank NV Amsterdam Sucursala București,"Art. 32 (1) b) GDPR, Art. 32 (2), (4) GDPR",Insufficient technical and organisational meas...
1,ETid-1967,SPAIN,2023-07-12,"10,000",Private individual,Art. 6 GDPR,Insufficient legal basis for data processing
2,ETid-1966,SPAIN,2023-07-16,300,Private individual,Art. 13 GDRP,Insufficient fulfilment of information obligat...
3,ETid-1965,IRELAND,2023-06-16,"1,000,000",Irish Departement of Health,"Art. 5 (1) c) GDPR, Art. 6 (1), (4) GDPR, Art....",Non-compliance with general data processing pr...
4,ETid-1964,SPAIN,2023-07-17,"10,000","NANDIVALE, S.L",Art. 6 (1) GDPR,Insufficient legal basis for data processing


In [14]:
total_df.shape

(1968, 7)

In [15]:
#check IDs unicity
if total_df['ID'].value_counts().sort_values(ascending = False)[0] < 2:
    print('No duplicates !')
else:
    print('Dedup to be done !')

No duplicates !


In [16]:
#export for saving
total_df.to_csv('export_GDPR_fines_last.csv')

## Data Cleaning

In [28]:
fines_df = pd.read_csv('export_GDPR_fines_last.csv').drop('Unnamed: 0', axis = 1)

In [29]:
display(fines_df.head())
display(fines_df.info())

,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type
0,ETid-1968,ROMANIA,2023-07-18,"3,000",ING Bank NV Amsterdam Sucursala București,"Art. 32 (1) b) GDPR, Art. 32 (2), (4) GDPR",Insufficient technical and organisational meas...
1,ETid-1967,SPAIN,2023-07-12,"10,000",Private individual,Art. 6 GDPR,Insufficient legal basis for data processing
2,ETid-1966,SPAIN,2023-07-16,300,Private individual,Art. 13 GDRP,Insufficient fulfilment of information obligat...
3,ETid-1965,IRELAND,2023-06-16,"1,000,000",Irish Departement of Health,"Art. 5 (1) c) GDPR, Art. 6 (1), (4) GDPR, Art....",Non-compliance with general data processing pr...
4,ETid-1964,SPAIN,2023-07-17,"10,000","NANDIVALE, S.L",Art. 6 (1) GDPR,Insufficient legal basis for data processing


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1968 entries, 0 to 1967
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    1968 non-null   object
 1   Country               1968 non-null   object
 2   Date                  1968 non-null   object
 3   Amount                1968 non-null   object
 4   Controller_Processor  1968 non-null   object
 5   Quoted_Art            1968 non-null   object
 6   Type                  1968 non-null   object
dtypes: object(7)
memory usage: 107.8+ KB


None

In [30]:
def prepare_data(df):
    
    df['ID'] = df['ID'].str.replace('ETid-', '')
    df['ID'] = df['ID'].astype(int)

    df['Amount'] = df['Amount'].str.replace(' ','').str.replace(r'[a-zA-Z,-]','', regex = True).replace(r'^\s*$', 0, regex=True)
    df['Amount'] = df['Amount'].fillna('0')
    df['Amount'] = df['Amount'].astype(int)
    
    df.loc[df['Amount'] == 0]['Amount'] = int(df['Amount'].median())
        
    df['Date'] = df['Date'].str.replace('Unknown','')
    df['Date'] = df['Date'].fillna(method = 'ffill')

    #df.loc[len(df['Date']) == 4]
        
    #treatment of article column
    df['Quoted_Art'] = df['Quoted_Art'].str.replace(r'[a-zA-Z.§é]','').str.replace(' ','').str.replace(r"\(.*\)",'')
    df = df.join(pd.get_dummies(df['Quoted_Art'].str.split(',').apply(pd.Series).stack(), prefix = 'Art').sum(level=0))    
    
    df['VIOLATED_ARTICLES'] = df[[x for x in df.columns if "Art" in x]].sum(axis = 1)
                        
    return df

In [31]:
data = prepare_data(fines_df)

C:\Users\valen\AppData\Local\Temp\ipykernel_25996\3030326006.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['Amount'] == 0]['Amount'] = int(df['Amount'].median())


TypeError: sum() got an unexpected keyword argument 'level'

In [21]:
data

[['ETid-1968',
  'ROMANIA',
  '2023-07-18',
  '3,000',
  'ING Bank NV Amsterdam Sucursala București',
  'Art. 32 (1) b) GDPR, Art. 32 (2), (4) GDPR',
  'Insufficient technical and organisational measures to ensure information security',
  'link'],
 ['ETid-1967',
  'SPAIN',
  '2023-07-12',
  '10,000',
  'Private individual',
  'Art. 6 GDPR',
  'Insufficient legal basis for data processing',
  'link'],
 ['ETid-1966',
  'SPAIN',
  '2023-07-16',
  '300',
  'Private individual',
  'Art. 13 GDRP',
  'Insufficient fulfilment of information obligations',
  'link'],
 ['ETid-1965',
  'IRELAND',
  '2023-06-16',
  '1,000,000',
  'Irish Departement of Health',
  'Art. 5 (1) c) GDPR, Art. 6 (1), (4) GDPR, Art. 9 (1) GDPR',
  'Non-compliance with general data processing principles',
  'link'],
 ['ETid-1964',
  'SPAIN',
  '2023-07-17',
  '10,000',
  'NANDIVALE, S.L',
  'Art. 6 (1) GDPR',
  'Insufficient legal basis for data processing',
  'link'],
 ['ETid-1963',
  'SPAIN',
  '2023-07-21',
  '720',
  '

In [56]:
data.to_csv('export_GDPR_fines_cleaned.csv')